In [1]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from isodate import parse_duration
import pyspark.sql.types as t
from typing import Tuple
import os

In [2]:
@f.udf(returnType=t.IntegerType())
def converter_segundos(tempo: str) -> int:
    """Converte para o total de seguntos

    Args:
        tempo (str): tempo

    Returns:
        int: total segundos
    """
    duracao = parse_duration(tempo)
    total_minutes = duracao.total_seconds() 
   
    return int(total_minutes)


In [3]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

23/11/02 01:16:43 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/02 01:16:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/02 01:16:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def carregar_dataframe_prata(spark_session : SparkSession, assunto: str, metrica: str = None):
    if metrica is not None:
        df_cities_skylines = spark_session \
            .read \
                .parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/{assunto}/extracao_data_*/{metrica}/*.parquet')
    else:
        df_cities_skylines = spark_session \
            .read \
                .parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/{assunto}/extracao_data_*/*.parquet')
    return df_cities_skylines

In [5]:
def criar_particao_ouro(dataframe: DataFrame, assunto_pesquisa: str, metrica: str,  nome_arquivo: str, colunas_particao: Tuple = None ):
    caminho_datalake_ouro = '/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/'
    if colunas_particao == None:
        dataframe.write. \
            parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))
    else: 
        if isinstance(colunas_particao, Tuple) :
            dataframe.write. \
                partitionBy(*colunas_particao). \
                parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))
        else:
            dataframe.write. \
                partitionBy(colunas_particao). \
                parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))


In [6]:
def transformacao_dataframe(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'data_extracao',
        f.explode('DATA_PUBLICACAO').alias('DATA_PUBLICACAO'),
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                f.explode('ID_CANAL').alias('ID_CANAL'),
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                f.explode('NM_CANAL').alias('NM_CANAL'), 
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                f.explode('ID_CATEGORIA').alias('ID_CATEGORIA'),
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                f.explode('ID_VIDEO').alias('ID_VIDEO'),
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                f.explode('TITULO_VIDEO').alias('TITULO_VIDEO'),
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'ID_VIDEO',
                f.explode('DESCRICAO').alias('DESCRICAO'),
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                f.explode('TAGS').alias('TAGS'),
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                f.explode('DURACAO_VIDEOS').alias('DURACAO_VIDEOS'),
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                f.explode('TOTAL_VISUALIZACOES').alias('TOTAL_VISUALIZACOES'),
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                f.explode('TOTAL_LIKES').alias('TOTAL_LIKES'),
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                f.explode('TOTAL_FAVORITOS').alias('TOTAL_FAVORITOS'),
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                f.explode('TOTAL_COMENTARIOS').alias('TOTAL_COMENTARIOS'),
        )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd"))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_SEGUNDOS', converter_segundos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_CARACTERE_VIDEO', f.length('TITULO_VIDEO'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_TAGS', f.when(f.size('TAGS') > 0, f.size('TAGS')).otherwise(0))
    return dataframe_transform
  

In [67]:
@f.udf(returnType=t.StringType())
def traduzir_dia_da_semana(dia):
    traducao = {
        'Sunday': 'Domingo',
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado'
    }
    return traducao.get(dia, dia)

In [68]:
@f.udf(returnType=t.IntegerType())
def indice_semana(dia):
    traducao = {
        'Domingo': 1,
        'Segunda-feira': 2,
        'Terça-feira': 3,
        'Quarta-feira': 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado': 7
    }
    return traducao.get(dia, dia)

In [69]:
traduzir_dia_da_semana('Sunday')

Column<'traduzir_dia_da_semana(Sunday)'>

In [70]:
df_estatisticas_teste = carregar_dataframe_prata(spark_session=spark, assunto='assunto_cities_skylines', metrica='estatisticas')
df_estatisticas_teste = transformacao_dataframe(df_estatisticas_teste)
df_estatisticas_teste.sort('ID_VIDEO').show(truncate=False)

+-------------+--------------------+------------------------+---------------+------------+-----------+---------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|data_extracao|DATA_PUBLICACAO     |ID_CANAL                |NM_CANAL       |ID_CATEGORIA|ID_VIDEO   |TITULO_VIDEO                                                                     |DESCRICAO                                           

<p>Depara Canal Vídeo</p>

In [71]:
def construir_depara_canal(dataframe: DataFrame, assunto: str):
    dataframe.select(
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO
    ).distinct().sort('ID_CANAL').write.\
        parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_{assunto}/depara_canal.parquet')


<p>O Número de comentários e likes gera mais views? </p>

In [72]:
def gerar_relacao_like_comentario_view_favoritos(dataframe: SparkSession):
    df_comentarios_likes = dataframe.select(
        dataframe.data_hora_extracao,
        dataframe.data_extracao,
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO,
        dataframe.TAGS,
        dataframe.TOTAL_VISUALIZACOES,
        dataframe.TOTAL_LIKES,
        dataframe.TOTAL_FAVORITOS,
        dataframe.TOTAL_COMENTARIOS
    ).filter(
        # (df_estatisticas.data_extracao == '2023-10-27') &
        (dataframe.TOTAL_VISUALIZACOES > 0) &
        (dataframe.TOTAL_LIKES > 0) 
    ).sort(f.col('TOTAL_VISUALIZACOES').desc())
    return df_comentarios_likes


In [73]:
lista_assunto = [
        'Power BI',
        'Python AND dados',
        'Cities Skylines',
        'Cities Skylines 2'
    ]

<p>ETL PARA OBTER RELAÇÃO LIKES - COMENTÁRIOS - VÍDEOS</p>

In [74]:

for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas')
    
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
   
    criar_particao_ouro(dataframe=df_estatisticas_etl, 
                        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='relacao_viusalicacao_comentario_likes',
        colunas_particao=('data_extracao', 'ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='relacao_viusalicacao_comentario_likes.parquet',
    )
    construir_depara_canal(dataframe=df_estatisticas_etl, assunto=assunto)

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


<p>Dia de publicação</p>

<p>Públicações por semana</p>

In [75]:
def obter_total_publicacao_video(dataframe : DataFrame) -> DataFrame:
   dataframe =  dataframe.select(
    'DATA_PUBLICACAO',
    'ID_VIDEO'
    ).withColumn(
       'DATA_PUBLICACAO', f.date_format('DATA_PUBLICACAO', 'yyyy-MM-dd')
    ) \
        .sort('ID_VIDEO') \
        .withColumn(
            'SEMANA_TRADUZIDA', 
            traduzir_dia_da_semana(
                f.date_format('DATA_PUBLICACAO', 'EEEE')
            )
            ).distinct() \
                .groupBy('DATA_PUBLICACAO','SEMANA_TRADUZIDA'). \
                    count(). \
                        alias('TOTAL_VIDEOS'). \
                            withColumn('INDICE_SEMANA', indice_semana('SEMANA_TRADUZIDA')) \
                    .sort('INDICE_SEMANA')\
                        .select('DATA_PUBLICACAO','SEMANA_TRADUZIDA', f.col('TOTAL_VIDEOS.COUNT').alias('TOTAL_VIDEOS'))       
   return dataframe

In [76]:
obter_total_publicacao_video(df_estatisticas_teste).show()

+---------------+----------------+------------+
|DATA_PUBLICACAO|SEMANA_TRADUZIDA|TOTAL_VIDEOS|
+---------------+----------------+------------+
|     2023-10-15|         Domingo|          12|
|     2023-10-22|         Domingo|          21|
|     2023-10-16|   Segunda-feira|          16|
|     2023-10-23|   Segunda-feira|          25|
|     2023-10-17|     Terça-feira|          18|
|     2023-10-24|     Terça-feira|          27|
|     2023-10-18|    Quarta-feira|          15|
|     2023-10-25|    Quarta-feira|          25|
|     2023-10-26|    Quinta-feira|          27|
|     2023-10-19|    Quinta-feira|          20|
|     2023-10-27|     Sexta-feira|          23|
|     2023-10-20|     Sexta-feira|          14|
|     2023-10-21|          Sábado|          25|
+---------------+----------------+------------+



<p>ETL Para obter o total de vídeos públicado por semaba</p>

In [77]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas'
        )
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    df_estatisticas_etl = obter_total_publicacao_video(df_estatisticas_etl)
    criar_particao_ouro(
        dataframe=df_estatisticas_etl, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='total_video_publicado_semana',
        nome_arquivo='total_video_publicado_semana.parquet',
    )
    
    

ETL PARA  Power BI


ETL PARA  Python AND dados
ETL PARA  Cities Skylines
ETL PARA  Cities Skylines 2


<p>Dia da semana com mais visualização</p>

In [78]:
df_estatisticas_teste.show()

+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|data_extracao|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|   ID_VIDEO|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS| data_hora_extracao|DURACAO_VIDEO_SEGUNDOS|TOTAL_CARACTERE_VIDEO|TOTAL_TAGS|
+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|   2023-10-27|2023-10-24T00

In [79]:
@f.udf(returnType=t.StringType())
def semana(indice: int) -> str:
    semana = {
        1: 'Domingo',
        2: 'Segunda-feira',
        3: 'Terça-feira',
        4: 'Quarta-feira',
        5: 'Quinta-feira',
        6: 'Sexta-feira',
        7: 'Sábado'
    }
    return semana.get(indice, indice)
    

<p>Vísualizações por semana</p>

In [80]:
df_v_teste = df_estatisticas_teste\
    .select('data_extracao', 'ID_CANAL','ID_VIDEO', 'TOTAL_VISUALIZACOES') \
    .withColumn('INDICE_SEMANA', f.dayofweek('data_extracao'))  \
    .withColumn('SEMANA', semana('INDICE_SEMANA')) \
    .groupBy('SEMANA', 'ID_CANAL', 'ID_VIDEO')\
        .max('TOTAL_VISUALIZACOES')\
        .sort('SEMANA')\
        .withColumnRenamed('max(TOTAL_VISUALIZACOES)', 'MAX_TOTAL_VISUALIZACOES')

df_v_testev2 = df_v_teste.withColumn('ISEMANA', indice_semana('SEMANA')) \
        .groupBy('ISEMANA', 'SEMANA') \
        .sum('MAX_TOTAL_VISUALIZACOES') \
        .sort('ISEMANA') \
    .withColumnRenamed('sum(MAX_TOTAL_VISUALIZACOES)', 'TOTAL_VISUALIZACOES')
df_v_testev2 = df_v_testev2.drop('ISEMANA')
df_v_testev2.show()

+-------------+-------------------+
|       SEMANA|TOTAL_VISUALIZACOES|
+-------------+-------------------+
|      Domingo|             525028|
|Segunda-feira|             670037|
|  Terça-feira|             799358|
| Quarta-feira|             928146|
| Quinta-feira|            1076002|
|  Sexta-feira|            1185093|
|       Sábado|             366474|
+-------------+-------------------+



In [81]:
def obter_total_visualizacoes_semana(dataframe: DataFrame) -> DataFrame:
    df_v_teste = dataframe\
        .select('data_extracao', 'ID_CANAL','ID_VIDEO', 'TOTAL_VISUALIZACOES') \
        .withColumn('INDICE_SEMANA', f.dayofweek('data_extracao'))  \
        .withColumn('SEMANA', semana('INDICE_SEMANA')) \
        .groupBy('data_extracao','SEMANA', 'ID_CANAL', 'ID_VIDEO')\
            .max('TOTAL_VISUALIZACOES')\
            .sort('SEMANA')\
            .withColumnRenamed('max(TOTAL_VISUALIZACOES)', 'MAX_TOTAL_VISUALIZACOES')

    df_v_testev2 = df_v_teste.withColumn('ISEMANA', indice_semana('SEMANA')) \
            .groupBy('data_extracao','ISEMANA','ID_CANAL','ID_VIDEO', 'SEMANA') \
            .sum('MAX_TOTAL_VISUALIZACOES') \
            .sort('ISEMANA') \
        .withColumnRenamed('sum(MAX_TOTAL_VISUALIZACOES)', 'TOTAL_VISUALIZACOES')
    df_v_testev2 = df_v_testev2.drop('ISEMANA')
    return df_v_testev2


In [82]:
obter_total_visualizacoes_semana(df_estatisticas_teste).show()

+-------------+--------------------+-----------+-------+-------------------+
|data_extracao|            ID_CANAL|   ID_VIDEO| SEMANA|TOTAL_VISUALIZACOES|
+-------------+--------------------+-----------+-------+-------------------+
|   2023-10-22|UCCe4Be21OPPltTTI...|gYl77Ot9oDU|Domingo|               2392|
|   2023-10-22|UCDGo_s0I-2JpO3o3...|C1s-hsI9JaY|Domingo|               4777|
|   2023-10-15|UC1mk6EtfMjxR4eEZ...|YhXTcceM1NA|Domingo|                181|
|   2023-10-22|UCp7gzkfbWE7jazP6...|tpBDkZWDK3o|Domingo|                484|
|   2023-10-22|UCABm0SdJCoM6z0MF...|Dpz3Gw_Wcd4|Domingo|                 89|
|   2023-10-22|UCayiH2Y5JBEOSubq...|FK-EYFdPqpA|Domingo|               3349|
|   2023-10-22|UCEc5jA0NvvR-yJC0...|igxEHvMej3g|Domingo|                 11|
|   2023-10-22|UC7v51W2NoLFawizg...|LiGVg86YIbM|Domingo|                  4|
|   2023-10-22|UCocGh-G6RCQZZWV9...|jVSSbjcwJj4|Domingo|                 20|
|   2023-10-22|UCcn60fs5EMW8A_k9...|8z6ulMkX86I|Domingo|                158|

<p>ETL Para visializações por semana<P>

In [83]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas'
        )
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    df_estatisticas_etl = obter_total_visualizacoes_semana(df_estatisticas_etl)
    criar_particao_ouro(
        dataframe=df_estatisticas_etl, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='total_visualizacao_por_semana',
        colunas_particao=('data_extracao','ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='total_visualizacao_por_semana.parquet',
    )

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


<p>Comentários</p>

In [84]:
df_comentarios = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_power_bi', 
        metrica='comentarios'
)

In [85]:
def extrair_comentarios(dataframe: DataFrame) -> DataFrame:
    dataframe = dataframe \
        .select('ID_CANAL', 'ID_COMENTARIO', 'ID_VIDEO', 'TEXTO_COMENTARIO')  \
        .distinct().sort('ID_VIDEO') 
    return dataframe
# Ugw-AOQ6Kh-Qof2U9bd4AaABAg - Gl60rzPP9VQ

In [86]:
extrair_comentarios(df_comentarios).show(truncate=False)

+------------------------+--------------------------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID_CANAL                |ID_COMENTARIO             |ID_VIDEO   |TEXTO_COMENTARIO                                                                                                                                                                                                   |
+------------------------+--------------------------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|UCg6yqeJJ2WMFVt9bXeUd-zA|UgwcmMNJM8lHN6YKgF94AaABAg|-Jg3j6EH7cg|Um tecnólogo com conhecimentos técnicos e proficiência em idiomas é suficiente, desde que também poss

<p>Extacao comentário</p>

In [87]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_comentarios = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='comentarios'
    )
    df_comentarios = extrair_comentarios(df_comentarios)
    criar_particao_ouro(
        dataframe=df_comentarios, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='comentarios',
        colunas_particao=('ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='comentarios.parquet',
    )

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


In [88]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()

    df_resposta_comentarios = carregar_dataframe_prata(
        assunto=f'assunto_{assunto}',
        spark_session=spark,
        metrica='resposta_comentarios'
    )   


    df_resposta_comentarios = df_resposta_comentarios.select(
        'ID_CANAL', 
        'ID_RESPOSTA_COMENTARIOS', 
        'TEXTO').withColumnRenamed(
            'ID_CANAL', 'ID_CANAL'
        ).distinct()
    

    # df_resposta_comentarios = df_resposta_comentarios \
    #     .select('ID_CANAL', 'ID_RESPOSTA_COMENTARIOS', 'TEXTO')\
    #     .withColumnRenamed('CABAK', "IDCANAL") \
    #     .distinct()
    # df_resposta_comentarios.show()
    criar_particao_ouro(
        dataframe=df_resposta_comentarios, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='reposta_comentarios',
        colunas_particao=('ID_CANAL', 'ID_RESPOSTA_COMENTARIOS'),
        nome_arquivo='reposta_comentarios.parquet',
    )

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


In [89]:
df_resposta_comentarios.select('ID_CANAL', 'ID_RESPOSTA_COMENTARIOS', 'TEXTO').distinct().show(truncate=False)

+------------------------+-------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID_CANAL                |ID_RESPOSTA_COMENTARIOS                          |TEXTO                                                                                                                                                                                                                                                                                                                         |
+------------------------+-------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------